In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer
import import_ipynb
from Statsdata import (get_freq_fitness, get_duration_fitness, get_time_fitness, get_activity_freq_stats, get_activity_duration_stats, 
get_freq_hour_normalized, get_freq_position_normalized, get_position_fitness, get_trace_length_stats, get_length_fitness)
from Mining import discovery_inductive, discovery_heuristic, conformance
import subprocess
import csv
import sys
import os
import uuid
import pandas as pd
from prettytable import PrettyTable as pt


importing Jupyter notebook from Mining.ipynb


In [11]:
def writeTable(stats, title, f):
        # Creating object
        tt = pt([""])
        # Adding rows
        for stat, values in stats.items():
                tb = pt()
                rows = []
                if(type(values) != dict):
                        tb.add_row([values])
                else:
                        for act, value in values.items():
                                tb.add_row([act, value])
                tt.add_row([tb.get_string(title=stat, header=False)])
        tt.align[""] = "l"
        
        f.write(tt.get_string(title=title, header=False) + "\n")

## Log import

In [12]:
# Just for testing
# logB = xes_importer.apply("../logNormal.xes")
# logA = xes_importer.apply("../logFreq.xes")

# path_logA = "../Example/Scenario1/logNormal.xes"
# path_logB = "../Example/Scenario1/logDur.xes"
# discovery("../Example/Scenario1/logNormal.xes", "../Example/Scenario1/logFreq.xes")


parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 4572.44it/s]


Discovering using Inductive...
Done
Discovering using Heuristic...
Done
Discovering of DCRgraph...
Discovering frequency stats...
Done
Discovering duration stats...
Done
Discovering time stats...
Done
Discovering frequency in positions...
Done
Discovering trace length stats...
{'Mean': 4.02, 'Median': 4.0, 'StDev': 1.6304165904020538, 'Min': 2, 'Max': 6}
Done
Conformance checking from Inductive...


aligning log, completed variants :: 100%|██████████| 3/3 [00:00<00:00, 598.39it/s]


Done
Conformance checking from Heuristic...


aligning log, completed variants :: 100%|██████████| 3/3 [00:00<00:00, 498.97it/s]

Done
Conformance of DCR...
Done
Conformance of Palia...


FileNotFoundError: [Errno 2] No such file or directory: 'paliacc.txt'

In [3]:
def execall(path_logA, path_logB):
    logA = xes_importer.apply(path_logA)
    logB = xes_importer.apply(path_logB)

    # Discovery using Inductive
    print("Discovering using Inductive...")
    pind, filename_ind = discovery_inductive(logB)
    print("Done")

    # Discovery using Heuristic
    print("Discovering using Heuristic...")
    pheu, filename_heu = discovery_heuristic(logA)
    print("Done")

    # Discovery of DCR
    # java -jar "dcr-discovery.jar" "path xes" "path JSON .JSON"
    print("Discovering of DCRgraph...")
    PATH = 'Models'
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    filepath_dcr = "Models\dcr_"+str(uuid.uuid4())+".JSON"
    subprocess.call(['java', '-jar', 'dcr-discovery.jar', path_logA, filepath_dcr])

    # Discovery of Palia
    # java -jar "palia-discovery.jar" "path xes"
    print("Discovering of Palia graph...")
    filepath_palia = "Models\palia_"+str(uuid.uuid4())+".JSON"
    subprocess.call(['java', '-jar', 'palia-discovery.jar', path_logA, filepath_palia])

    f = open("Model\Discovery_stats_"+str(uuid.uuid4())+".txt", "w")

    # Discovery of freq stats
    print("Discovering frequency stats...")
    fstats, attrvalues = get_activity_freq_stats(logA)
    writeTable(fstats, "Frequency", f)
    # fsum, fmean, fmedian, fstdev, fmin, fmax
    print("Done")

    # Discovery of duration stats
    print("Discovering duration stats...")
    dstats = get_activity_duration_stats(logA)
    if(dstats is None):
        print("No timestamp attribute defined")
    else: 
        writeTable(dstats, "Duration", f)
    #  dmean, dmedian, dmin, dmax, dstdev
    print("Done")

    # Discovery of absolute time stats
    print("Discovering time stats...")
    abstime = get_freq_hour_normalized(logA)
    if(abstime is None):
        print("No timestamp attribute defined")
    else: 
        writeTable(abstime, "Absolute time", f)
    print("Done")

    # Discovery of freq position stats
    print("Discovering frequency in positions...")
    posfreq = get_freq_position_normalized(logA)
    writeTable(posfreq, "Position frequency", f)
    print("Done")

    # Discovery of traces length stats
    print("Discovering trace length stats...")
    lenstats = get_trace_length_stats(logA)
    writeTable(lenstats, "Trace length", f)
    print("Done")

    f.close()
    # with open('Discovery.csv', 'w', newline="") as f:
    #     writer = csv.writer(f)
    #     writer.writerow(["Frequency statistics"])
    #     for stat in fstats:
    #         writer.writerow(stat)
    #     print("Done")

    ##################################################################
    ##################################################################
    # Conformance checking (alignment) from Inductive
    print("Conformance checking from Inductive...")
    ccind_traces, ccind = conformance(filename_ind, logA, logB)
    print("Done")

    # Conformance checking (alignment) from Heuristic
    print("Conformance checking from Heuristic...")
    ccheu_traces, ccheu = conformance(filename_heu, logA, logB)
    print("Done")

    # Conformance checking of DCR
    # java -jar "dcr-conformance.jar" "path model .JSON" "path logB" open world flag
    print("Conformance of DCR...")
    subprocess.call(['java', '-jar', 'dcr-conformance.jar', filepath_dcr, path_logB, "FALSE"])

    # The total fitness value is saved in the file dcrcc.txt
    f = open("dcrcc.txt", "r")
    for line in f:
        ccdcr = float(line)
    f.close()
    print("Done")

    # Conformance checking of Palia
    # java -jar "palia-conformance.jar" "path model .JSON" "path logB"
    print("Conformance of Palia...")
    subprocess.call(['java', '-jar', 'palia-conformance.jar', filepath_palia, path_logB])

    # The total fitness value is saved in the file paliacc.txt
    f = open("paliacc.txt", "r")
    for line in f:
        ccpalia = float(line)
    f.close()
    print("Done")

    # Conformance of frequency
    print("Conformance of frequency...")
    ccfreq_ev, ccfreq_traces, ccfreq = get_freq_fitness(fstats, attrvalues, logB)
    print("Done")

    # Conformance of duration
    if(not dstats is None):
        print("Conformance of duration...")
        ccdur_ev, ccdur_traces, ccdur = get_duration_fitness(dstats, attrvalues, logB)
        print("Done")
    else: ccdur = ""

    # Get fitness of absolute time
    if(not abstime is None):
        print("Absolute time comparison...")
        cctime_act, cctime = get_time_fitness(abstime, logB)
        print("Done")
    else: cctime = ""

    # Get fitness of events positions
    print("Position frequency comparison...")
    ccpos_act, ccpos = get_position_fitness(posfreq, logB)
    print("Done")

    # Get fitness of trace length
    print("Trace length comparison...")
    cclen_traces, cclen = get_length_fitness(lenstats, logB)
    print("Done")

    # ccfreq 
    # ccdur 
    # ccheu
    # ccind
    # ccdcr
    # ccdecl

    header = ["logA", "logB", "CCHeu", "CCInd", "CCDcr","CCPalia", "CCFreq", "CCDur", "CCTime", "CCPos", "CCLen"]
    values = [path_logA, path_logB, ccheu, ccind, ccdcr, ccpalia, ccfreq, ccdur, cctime, ccpos, cclen]
    print("Writing the results...")
    # open the file in the write mode

    # Create File
    if not os.path.exists('Results.csv'):
        print("Creating file...")
        with open('Results.csv', 'w', newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerow(values)
            print("Done")
    else:
        with open('Results.csv', 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow(values)
            print("Done")


In [ ]:
A = sys.argv[1]
for i in range(2, len(sys.argv)):
    B = sys.argv[i] 
    execall(A,B)
    print("####################")
    print("TASKS COMPLETED")
    print("####################")